In [2]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd
from vectorbtpro import *

# Custom Data class to load symbol data from a local pickle file
class DBData(vbt.Data):
    @classmethod
    def fetch_symbol(cls, symbols, **kwargs):
        # fetch_symbol can handle multiple symbols
        df = get_db_data(symbols, **kwargs)
        return df

# Function to read data from pickle file
def get_db_data(file_name, **filters):
    df = pd.read_pickle(file_name)
    return df

In [3]:


# Pull data using custom DBData class
db_data = DBData.pull(
    'NetGEX_AbsGEX_EPS(SPY).pickle',
    silence_warnings=True
)

# Extract price and GEX-related series
spot_close = db_data.get('Spot_Close')
net_gex    = db_data.get('close_net_gex')
abs_gex    = db_data.get('close_abs_gex')

# Price series used for signal generation
price = spot_close

In [4]:


# Generate basic signals
price_up = (price > price.vbt.fshift())                        # Price moved up from previous bar
net_gex_down = (net_gex < net_gex.vbt.fshift())                # Net GEX dropped
price_flat = (price.vbt.fshift() - price).abs() < 0.001        # Price change < 0.1%
abs_gex_rising = (abs_gex > abs_gex.vbt.fshift())              # Absolute GEX increasing

# Logic combinations for strategy signals
short_entry = (price_up & net_gex_down).astype(bool)           # Reversal / short pressure
long_entry = (price_flat & abs_gex_rising).astype(bool)        # Coiled signal

In [5]:


# Shift signals forward to simulate execution on next bar
long_entry = long_entry.vbt.signals.fshift()
short_entry = short_entry.vbt.signals.fshift()

In [6]:


# Backtest with vectorbt Portfolio
pf = vbt.Portfolio.from_signals(
    close=price,
    entries=long_entry,
    exits=None,
    short_entries=short_entry,
    short_exits=None,
    init_cash=100_000,
    size=0.1,                            # 10% of portfolio per trade
    size_type="valuepercent"            # Size by % of portfolio value
)


In [7]:

# Print backtest stats
pf.stats()


Start Index                   2023-04-03 13:36:00+00:00
End Index                     2024-12-20 20:59:00+00:00
Total Duration                        115 days 15:00:00
Start Value                                    100000.0
Min Value                                  96382.352613
Max Value                                 100256.387524
End Value                                   96660.86593
Total Return [%]                              -3.339134
Benchmark Return [%]                          44.205906
Position Coverage [%]                         99.998799
Max Gross Exposure [%]                        10.290141
Max Drawdown [%]                               3.864128
Max Drawdown Duration                  76 days 17:59:00
Total Orders                                       3701
Total Fees Paid                                     0.0
Total Trades                                       3701
Win Rate [%]                                  49.405405
Best Trade [%]                                 5

In [8]:

# Plot portfolio equity and trades
pf.plot()


FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'mode': 'lines',
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '1e5ad97b-3beb-4898-8d91-bcfbe303eec3',
              'x': array([datetime.datetime(2023, 4, 3, 13, 36, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 4, 3, 13, 37, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 4, 3, 13, 38, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 20, 20, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 20, 20, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 20, 20, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array([409.7405, 409.7

In [9]:

# --- Additional Analysis ---

# Reversal signal: Large up move + Net GEX down (short pressure)
price_move = (price - price.vbt.fshift()).abs()
reversal_signal = (price_move > 0.2) & (price > price.vbt.fshift()) & (net_gex < net_gex.vbt.fshift())
reversal_signal = reversal_signal.astype(bool)

In [10]:


# Coiled move signal: Price flat, Abs GEX rising
price_flat = (price - price.vbt.fshift()).abs() < 0.1
abs_gex_rising = abs_gex > abs_gex.vbt.fshift()
coiled_signal = (price_flat & abs_gex_rising).astype(bool)


In [11]:

# Plot Price vs Net GEX with signal overlay
fig = price.vbt.plot(title='Price vs Net GEX Reversal Signal')

# Add Net GEX on secondary y-axis
net_gex.vbt.plot(
    fig=fig,
    trace_kwargs=dict(
        yaxis='y2',
        name='Net GEX',
        line=dict(color='orange')
    )
)

fig.update_layout(
    yaxis2=dict(
        title='Net GEX',
        overlaying='y',
        side='right',
        showgrid=False
    )
)

# Plot reversal signals as red arrow markers
reversal_signal.vbt.signals.plot_as_markers(
    y=price,
    fig=fig,
    trace_kwargs=dict(
        marker=dict(
            color='red',
            size=10,
            symbol='arrow-up'
        ),
        name='Reversal Signal'
    )
)


FigureWidget({
    'data': [{'name': 'Spot_Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'f2a35e56-1237-4313-8251-5e6961cb24cd',
              'x': array([datetime.datetime(2023, 4, 3, 13, 36, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 4, 3, 13, 37, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 4, 3, 13, 38, tzinfo=datetime.timezone.utc),
                          ...,
                          datetime.datetime(2024, 12, 20, 20, 57, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 20, 20, 58, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2024, 12, 20, 20, 59, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'y': array([409.7405, 409.77  , 409.81  , ..., 589.76  , 590.11  , 590.87  ])},
             {'line': {'color': 'orange'},
              'name': 'Net GEX',
   